In [1]:
from LIMxCMBL.init import *
from LIMxCMBL.kernels import *
import sys

# Lambda_idx = int(sys.argv[1])
# n_external = int(sys.argv[2])
# ell_idx = int(sys.argv[3])


Lambda_idx = 18
n_external = 200
ell_idx = 0

Lambda = Lambdas[Lambda_idx]

# CCAT-prime
zmin = 3.5
zmax = 8.1

oup_fname = '/scratch/users/delon/LIMxCMBL/I_auto/comb_zmin_%.1f_zmax_%.1f_Lambda_idx_%d_n_ext_%d_l_%d_quad.npy'%(zmin, zmax, 
                                                                                                                     Lambda_idx, 
                                                                                                                     n_external,
                                                                                                                    ell_idx)
print(oup_fname)

/scratch/users/delon/LIMxCMBL/I_auto/comb_zmin_3.5_zmax_8.1_Lambda_idx_18_n_ext_200_l_0_quad.npy


In [2]:
Omega_field = 8 * (np.pi/180)**2 #rad^2
Pei = 2.3e4 #Mpc^3 kJy^2 /sr^2 
chimin = ccl.comoving_angular_distance(cosmo, 1/(1+zmin))
chimax = ccl.comoving_angular_distance(cosmo, 1/(1+zmax))

In [3]:
from scipy.interpolate import interp1d, LinearNDInterpolator
from scipy.integrate import quad, quad_vec, trapezoid

external_chis = np.linspace(chimin*(1+1e-8), chimax*(1 - 1e-8), n_external)
print('external chi spacing', np.mean(np.diff(external_chis)))

inner_dkparp_integral = np.load('/oak/stanford/orgs/kipac/users/delon/LIMxCMBL/inner_dkparp_integral.npy')
tmp_chibs = []
tmp_log_deltas = []
tmp_fnctn = []
for i in range(len(chibs)):
    for j in range(len(deltas)):
        tmp_chibs += [chibs[i]]
        tmp_log_deltas += [np.log10(deltas[j])]
        tmp_fnctn += [inner_dkparp_integral[ell_idx,i,j]]
        
f_inner_integral = LinearNDInterpolator(list(zip(tmp_chibs, tmp_log_deltas)), tmp_fnctn)
f_inner_integral_LoLo = interp1d(x = chibs, y = inner_dkparp_integral[ell_idx], axis = 0)

external chi spacing 11.103205339966282


In [4]:
from tqdm import trange

# for LoLo

f_KLIMLo1 = get_f_KILo(external_chi = external_chis.reshape(n_external,1,1), Lambda=Lambda)
f_KLIMLo1_windowed = apply_window(f_K = f_KLIMLo1,
                                 chimin = chimin,
                                 chimax = chimax)

f_KLIMLo2 = get_f_KILo(external_chi = external_chis.reshape(1,n_external, 1), Lambda=Lambda)
f_KLIMLo2_windowed = apply_window(f_K = f_KLIMLo2,
                                 chimin = chimin,
                                 chimax = chimax)

#for cross

f_KLIM   = get_f_KI()

f_KLIM_windowed = apply_window(f_K = f_KLIM,
                               chimin = chimin,
                               chimax = chimax)
f_KLIMLo = get_f_KILo(external_chi = external_chis, Lambda=Lambda)

f_KLIMLo_windowed = apply_window(f_K = f_KLIMLo,
                                 chimin = chimin,
                                 chimax = chimax)



def f_integrand(_chib):
    
    #I Lo + Lo I
    _delta = np.abs(1 - external_chis/_chib) #(n_ext)
    _delta = np.where(_delta < 1e-6, 1e-6, 
                     np.where(_delta > 0.7, 0.7, _delta))
    
    cross_integrand = 2 * f_KLIM_windowed(external_chis) #chi (n_ext)
    cross_integrand *= f_inner_integral([(_chib, np.log10(_d)) for _d in _delta]) #(n_ext)
    cross_integrand /= (_chib**2) #(n_ext)
    cross_integrand = np.einsum('x,xy->xy', 
                           cross_integrand, 
                           f_KLIMLo_windowed((2*_chib - external_chis.reshape(n_external, 1))))
    cross_integrand = cross_integrand + np.moveaxis(cross_integrand, -1, -2) # the two cross terms are just from switching chi and chi'

    
    #LoLo
    plus = _chib*(1+deltas)
    mins = _chib*(1-deltas)
    LoLo_integrand  = f_KLIMLo1_windowed(plus) * f_KLIMLo2_windowed(mins) #(x,y,d)
    LoLo_integrand += f_KLIMLo1_windowed(mins) * f_KLIMLo2_windowed(plus) #(x,y,d)
    LoLo_integrand *= (2 / _chib) #(x,y,d)
    LoLo_integrand = np.einsum('xyd,d->xyd', LoLo_integrand, deltas)
    LoLo_integrand = np.einsum('xyd,d->xyd', LoLo_integrand, f_inner_integral_LoLo(_chib))
    LoLo_integrand = trapezoid(x = np.log(deltas), y = LoLo_integrand, axis=-1)
    
    return LoLo_integrand - cross_integrand

In [5]:
res, _ = quad_vec(f_integrand, 10, chimax_sample, epsrel = 1e-3, epsabs =0.0, workers=32)

In [6]:
np.save(oup_fname, res)
print('outputted')

outputted
